# Synthetic Cities MVP

here some exaplaination

## 1. Load Packages

In [2]:
import argparse
import random
import numpy as np
import logging
import datetime

import torch
import torch.nn as nn
# import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import sys
sys.path.append("../src/models/")
from Generator import OptGen

import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

from IPython.display import HTML

import optuna
from optuna.trial import TrialState

import mlflow

logging.basicConfig(filename='../reports/MVPOptimizer.log')

manualSeed = 999
# manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

%load_ext autoreload
%autoreload 2
%autosave 0

Random Seed:  999
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosave disabled


# 2. Set parameters

In [3]:
dataroot = "../data/cadastralExport" # Root directory for train dataset
datarootTest = "../data/cadastralExport/" # Root directory for dataset
workers = 2 # Number of workers for dataloader
batch_size = 128 # Batch size during training
image_size = 64 # Spatial size of training images. All images will be resized to this
nc = 3 # Number of channels in the training images. For color images this is 3
nz = 100 # Size of z latent vector (i.e. size of generator input)
ngf = 64 # Size of feature maps in generator
ndf = 64 # Size of feature maps in discriminator
num_epochs = 50 # Number of training epochs
lr = 0.0002 # Learning rate for optimizers
beta1 = 0.5 # Beta1 hyperparam for Adam optimizers
ngpu = torch.cuda.device_count() # Number of GPUs available. Use 0 for CPU mode.
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

## 3. Create Dataloader

In [ ]:
# We can use an image folder dataset the way we have it setup.
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               # transforms.Resize(image_size*4),
                               transforms.RandomCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)));

### WARNING: only a small part of data is loaded

In [ ]:
dataset

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(dataroot) if isfile(join(dataroot, f))]
len(onlyfiles)

## 4. Weight Initialization 

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02) # fills the weights? gamma param with normal distribution
        nn.init.constant_(m.bias.data, 0) # fills the bias with the constant 0

## 5. Import Generator

In [ ]:
n_covs = random.randint(3,6)
drop_convs2d = random.uniform(0.4,0.6)

GNet = OptGen(ngpu, n_covs, drop_convs2d).to(device)
# no data parallelization needed, since there is one gpu
GNet.apply(weights_init)
print(GNet)

## 6. Discriminator

In [ ]:
s = [10, 9, 87, 5 ,8]
np.round(np.mean(s),0
        )

In [ ]:
np.clip(10,9,10)